From: https://w.wiki/6cs5

In [1]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

from PIL import Image
import requests

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?painting ?paintingLabel ?artist ?artistLabel ?originLabel ?image WHERE {
  ?painting wdt:P31 wd:Q3305213;
    wdt:P170 ?artist;
    wdt:P276 ?origin.
  ?artist wdt:P27 wd:Q38.
  ?origin wdt:P17 wd:Q38.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "de". }
  OPTIONAL { ?painting wdt:P18 ?image. }
}
LIMIT 9"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    print(result)


{'artist': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q40823'}, 'painting': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q28803748'}, 'image': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Grechetto%20%28Giovanni%20Benedetto%20Castiglione%29%20-%20Crucifixion%20-%20Google%20Art%20Project.jpg'}, 'paintingLabel': {'type': 'literal', 'value': 'Q28803748'}, 'artistLabel': {'xml:lang': 'de', 'type': 'literal', 'value': 'Giovanni Benedetto Castiglione'}, 'originLabel': {'xml:lang': 'de', 'type': 'literal', 'value': 'Musei di Strada Nuova'}}
{'artist': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q40823'}, 'painting': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q28803749'}, 'image': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Grechetto%20%28Giovanni%20Benedetto%20Castiglione%29%20-%20Journey%20of%20the%20Family%20of%20Abraham%20-%20Google%20Art%20Project.jpg'}, 'paintingLabel': {